In [218]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
from time import sleep


In [123]:
# Cấu hình tùy chọn cho Edge
options = Options()
options.add_argument("--start-maximized")  # Mở ở chế độ toàn màn hình
options.add_argument("--inprivate")  # Chế độ ẩn danh (InPrivate mode)

In [124]:
driver = webdriver.Edge(options=options)
driver.get("https://www.facebook.com/groups/ued.confessions?locale=vi_VN")
username = driver.find_element(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .xod5an3 input')
password = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg input')[0]
login = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg')[1]
username.send_keys('khoalolriot@gmail.com')
password.send_keys('Akhoa@6204')
login.click()

In [180]:
def scroll(driver): 
    # Lấy chiều cao của trang hiện tại
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Lặp để cuộn trang xuống và kiểm tra
    while True:
        # Cuộn trang xuống cuối cùng
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Đợi cho đến khi một phần tử mới xuất hiện, ví dụ: bài đăng mới hoặc một phần tử trong danh sách
        sleep(3)
        
        # Lấy chiều cao của trang sau khi cuộn
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Kiểm tra nếu chiều cao của trang không thay đổi, tức là đã cuộn hết trang
        if new_height == last_height:
            print("Đã cuộn đến cuối trang!")
            break
        
        # Cập nhật chiều cao mới
        last_height = new_height

In [ ]:
def scroll_to_post(driver, post):
    driver.execute_script("arguments[0].scrollIntoView(true);", post)


In [241]:
scroll(driver)
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
print('Thu thập được:',len(post_list),'bài đăng')

Đã cuộn đến cuối trang!
Thu thập được: 648 bài đăng


In [ ]:
count = 0
ele = '.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd'
df = pd.DataFrame(columns=['linkPoster', 'name', 'time', 'content', 'content_img','status'])

for index, post in enumerate(post_list):
    try:
        scroll_to_post(driver, post)
        print(f"Đã scroll tới bài đăng số {index + 1}:")
        
        user_info = post.find_elements(By.CSS_SELECTOR, '.x1rg5ohu')
        
        user_link = post.find_elements(By.CSS_SELECTOR, '.xu06os2.x1ok221b a')[0].get_attribute('href')
        
        user_name = post.find_elements(By.CSS_SELECTOR, '.xu06os2.x1ok221b')[0].text
        
        time_post = post.find_elements(By.CSS_SELECTOR, '.xu06os2.x1ok221b')[1]

        if time_post.text.strip() == '·': 
            time_link = time_post.find_element(By.CSS_SELECTOR, 'use').get_attribute('xlink:href')
            time_post = driver.find_element(By.CSS_SELECTOR, f'text{time_link}').get_attribute('textContent')
        else: 
            time_post = time_post.text.replace('·','').strip()
            
        content_post = post.find_element(By.CSS_SELECTOR, '[data-ad-rendering-role="message"]')
        try: 
            more_content = content_post.find_element(By.CSS_SELECTOR, 'div[role="button"]').click()
            content_post = post.find_element(By.CSS_SELECTOR, '[data-ad-rendering-role="message"]').text
        except NoSuchElementException: 
            content_post = post.find_element(By.CSS_SELECTOR, '[data-ad-rendering-role="message"]').text
        
        # 1 ảnh
        content_img_post = {}        
        i = 0
        try:     
            content_img_post[i] = post.find_element(By.CSS_SELECTOR, '.x6s0dn4.x1jx94hy.x78zum5.xdt5ytf.x6ikm8r.x10wlt62.x1n2onr6.xh8yej3 img').get_attribute('src')
            i += 1
        except NoSuchElementException: 
            pass
        
        # nhiều ảnh
        try:     
            content_img_list = post.find_elements(By.CSS_SELECTOR, '.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x6ikm8r.x10wlt62 img')
            for img in content_img_list:
                content_img_post[i] = img.get_attribute('src')
                i += 1 
        except NoSuchElementException: 
            pass 

        post_data = {
            'linkPoster': user_link if user_name != 'Người tham gia ẩn danh' else 'No link',
            'name': user_name,
            'time': time_post,
            'content': content_post if content_post else 'No content',
            'content_img': content_img_post if content_img_post else 'No picture',
            'status': 'Success'
        }
    except Exception: 
        post_data = {
            'linkPoster': 'error',
            'name': 'error',
            'time': 'error',
            'content': 'error',
            'content_img': 'error',
            'status': 'error'
        }
    df.loc[len(df)] = post_data





Đã scroll tới bài đăng số 1:
Đã scroll tới bài đăng số 2:
Đã scroll tới bài đăng số 3:
Đã scroll tới bài đăng số 4:
Đã scroll tới bài đăng số 5:
Đã scroll tới bài đăng số 6:
Đã scroll tới bài đăng số 7:
Đã scroll tới bài đăng số 8:
Đã scroll tới bài đăng số 9:
Đã scroll tới bài đăng số 10:
Đã scroll tới bài đăng số 11:
Đã scroll tới bài đăng số 12:
Đã scroll tới bài đăng số 13:
Đã scroll tới bài đăng số 14:
Đã scroll tới bài đăng số 15:
Đã scroll tới bài đăng số 16:
Đã scroll tới bài đăng số 17:
Đã scroll tới bài đăng số 18:
Đã scroll tới bài đăng số 19:
Đã scroll tới bài đăng số 20:
Đã scroll tới bài đăng số 21:
Đã scroll tới bài đăng số 22:
Đã scroll tới bài đăng số 23:
Đã scroll tới bài đăng số 24:
Đã scroll tới bài đăng số 25:
Đã scroll tới bài đăng số 26:
Đã scroll tới bài đăng số 27:
Đã scroll tới bài đăng số 28:
Đã scroll tới bài đăng số 29:
Đã scroll tới bài đăng số 30:
Đã scroll tới bài đăng số 31:
Đã scroll tới bài đăng số 32:
Đã scroll tới bài đăng số 33:
Đã scroll tới bài đ

In [276]:
# In ra các chỉ mục có 'status' là 'error'
for index in df[df['status'] == 'error'].index:
    print('Bài đăng bị lỗi:',index)

Bài đăng bị lỗi: 351
Bài đăng bị lỗi: 594
Bài đăng bị lỗi: 626
Bài đăng bị lỗi: 627
Bài đăng bị lỗi: 645
Bài đăng bị lỗi: 647


In [277]:
# Xuất kết quả ra file CSV
df.to_csv('D:/kpw/fb2/post_facebook.csv', index=False, encoding='utf-8')